In [1]:
import pandas as pd
import numpy as np

data_q4 = pd.read_excel('女胎检测数据.xlsx')
data_q4

,序号,孕妇代码,年龄,身高,体重,末次月经,IVF妊娠,检测日期,检测抽血次数,检测孕周,...,Unnamed: 21,X染色体浓度,13号染色体的GC含量,18号染色体的GC含量,21号染色体的GC含量,被过滤掉读段数的比例,染色体的非整倍体,怀孕次数,生产次数,胎儿是否健康
0,1,B001,32,162.0,82.0,2023-08-06 00:00:00,自然受孕,20231119,1,13w+5,...,NaN,-0.016976,0.379614,0.392832,0.402970,0.024708,NaN,1,0,是
1,2,B001,32,162.0,82.0,2023-08-06 00:00:00,自然受孕,20231208,2,17w+1,...,NaN,-0.024409,0.377950,0.391176,0.399509,0.025608,NaN,1,0,是
2,3,B001,32,162.0,85.0,2023-08-06 00:00:00,自然受孕,20240101,3,19w+6,...,NaN,-0.016357,0.385021,0.397515,0.408611,0.024452,NaN,1,0,是
3,4,B001,32,162.0,86.0,2023-08-06 00:00:00,自然受孕,20240123,4,23w,...,NaN,-0.021307,0.376070,0.389126,0.397539,0.022412,NaN,1,0,是
4,5,B002,33,165.0,96.0,2023-02-15 00:00:00,自然受孕,20230519,1,12w+5,...,NaN,-0.000886,0.378697,0.391196,0.401632,0.022843,NaN,1,0,是
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,601,B146,34,164.0,90.0,2022-12-31 00:00:00,自然受孕,20230615,4,23w+1,...,NaN,-0.014243,0.381691,0.393509,0.402032,0.019539,T18,≥3,1,是
601,602,B147,32,166.0,86.0,2023-01-08 00:00:00,自然受孕,20230423,1,15w,...,NaN,-0.000855,0.382151,0.395193,0.406132,0.022342,NaN,≥3,1,是
602,603,B147,32,166.0,84.0,2023-01-08 00:00:00,自然受孕,20230513,2,15w+4,...,NaN,-0.006456,0.378540,0.391420,0.400972,0.027777,NaN,≥3,1,是
603,604,B147,32,166.0,86.0,2023-01-08 00:00:00,自然受孕,20230610,3,19w+4,...,NaN,-0.013719,0.379481,0.391826,0.401168,0.022127,NaN,≥3,1,是


In [2]:
# 新增怀孕时间（天）列
def week2day(week: str):
    try:
        w = 0
        d = 0
        if '+' in week:
            WAD = week.split("+")
            w = int(WAD[0].strip("w").strip())
            d = int(WAD[1].strip("d").strip())
        elif 'w' in week:
            w = int(week.strip('w').strip())
            d = 0
        elif 'd' in week:
            w = 0
            d = int(week.strip('d').strip())
        else:
            # 如果格式不符合预期，返回NaN或默认值
            return pd.NaT
        return 7*w+d
    except (ValueError, AttributeError):
        # 处理转换错误
        return pd.NaT
    
    
data_q4["怀孕时间"] = data_q4["检测孕周"].apply(week2day)
data_q4

,序号,孕妇代码,年龄,身高,体重,末次月经,IVF妊娠,检测日期,检测抽血次数,检测孕周,...,X染色体浓度,13号染色体的GC含量,18号染色体的GC含量,21号染色体的GC含量,被过滤掉读段数的比例,染色体的非整倍体,怀孕次数,生产次数,胎儿是否健康,怀孕时间
0,1,B001,32,162.0,82.0,2023-08-06 00:00:00,自然受孕,20231119,1,13w+5,...,-0.016976,0.379614,0.392832,0.402970,0.024708,NaN,1,0,是,96
1,2,B001,32,162.0,82.0,2023-08-06 00:00:00,自然受孕,20231208,2,17w+1,...,-0.024409,0.377950,0.391176,0.399509,0.025608,NaN,1,0,是,120
2,3,B001,32,162.0,85.0,2023-08-06 00:00:00,自然受孕,20240101,3,19w+6,...,-0.016357,0.385021,0.397515,0.408611,0.024452,NaN,1,0,是,139
3,4,B001,32,162.0,86.0,2023-08-06 00:00:00,自然受孕,20240123,4,23w,...,-0.021307,0.376070,0.389126,0.397539,0.022412,NaN,1,0,是,161
4,5,B002,33,165.0,96.0,2023-02-15 00:00:00,自然受孕,20230519,1,12w+5,...,-0.000886,0.378697,0.391196,0.401632,0.022843,NaN,1,0,是,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,601,B146,34,164.0,90.0,2022-12-31 00:00:00,自然受孕,20230615,4,23w+1,...,-0.014243,0.381691,0.393509,0.402032,0.019539,T18,≥3,1,是,162
601,602,B147,32,166.0,86.0,2023-01-08 00:00:00,自然受孕,20230423,1,15w,...,-0.000855,0.382151,0.395193,0.406132,0.022342,NaN,≥3,1,是,105
602,603,B147,32,166.0,84.0,2023-01-08 00:00:00,自然受孕,20230513,2,15w+4,...,-0.006456,0.378540,0.391420,0.400972,0.027777,NaN,≥3,1,是,109
603,604,B147,32,166.0,86.0,2023-01-08 00:00:00,自然受孕,20230610,3,19w+4,...,-0.013719,0.379481,0.391826,0.401168,0.022127,NaN,≥3,1,是,137


In [4]:
new_columns = {
    'T13异常': np.nan,
    'T18异常': np.nan,
    'T21异常': np.nan,
    '二分类异常': np.nan,
    'T13T18异常': np.nan,
    'T13T21异常': np.nan,
    'T18T21异常': np.nan
}

# 使用 assign 方法批量添加空列
data_q4 = data_q4.assign(**new_columns)
data_q4

,序号,孕妇代码,年龄,身高,体重,末次月经,IVF妊娠,检测日期,检测抽血次数,检测孕周,...,生产次数,胎儿是否健康,怀孕时间,T13异常,T18异常,T21异常,二分类异常,T13T18异常,T13T21异常,T18T21异常
0,1,B001,32,162.0,82.0,2023-08-06 00:00:00,自然受孕,20231119,1,13w+5,...,0,是,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,B001,32,162.0,82.0,2023-08-06 00:00:00,自然受孕,20231208,2,17w+1,...,0,是,120,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,B001,32,162.0,85.0,2023-08-06 00:00:00,自然受孕,20240101,3,19w+6,...,0,是,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,B001,32,162.0,86.0,2023-08-06 00:00:00,自然受孕,20240123,4,23w,...,0,是,161,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,B002,33,165.0,96.0,2023-02-15 00:00:00,自然受孕,20230519,1,12w+5,...,0,是,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,601,B146,34,164.0,90.0,2022-12-31 00:00:00,自然受孕,20230615,4,23w+1,...,1,是,162,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,602,B147,32,166.0,86.0,2023-01-08 00:00:00,自然受孕,20230423,1,15w,...,1,是,105,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,603,B147,32,166.0,84.0,2023-01-08 00:00:00,自然受孕,20230513,2,15w+4,...,1,是,109,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603,604,B147,32,166.0,86.0,2023-01-08 00:00:00,自然受孕,20230610,3,19w+4,...,1,是,137,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# 先把空值填成空字符串，方便后面统一处理
data_q4['染色体的非整倍体'] = data_q4['染色体的非整倍体'].fillna('')

# 构造三个新列，默认 0
for col in ['T13异常', 'T18异常', 'T21异常']:
    data_q4[col] = 0

# 根据 AB 列内容打 1
data_q4.loc[data_q4['染色体的非整倍体'].str.contains('T13', na=False), 'T13异常'] = 1
data_q4.loc[data_q4['染色体的非整倍体'].str.contains('T18', na=False), 'T18异常'] = 1
data_q4.loc[data_q4['染色体的非整倍体'].str.contains('T21', na=False), 'T21异常'] = 1

# 更新“二分类异常”列
data_q4['二分类异常'] = np.where((data_q4['T13异常'] == 1) | (data_q4['T18异常'] == 1) | (data_q4['T21异常'] == 1), 1, 0)

# 更新“T13T18异常”列
data_q4['T13T18异常'] = np.where((data_q4['T13异常'] == 1) & (data_q4['T18异常'] == 1), 1, 0)

# 更新“T13T21异常”列
data_q4['T13T21异常'] = np.where((data_q4['T13异常'] == 1) & (data_q4['T21异常'] == 1), 1, 0)

# 更新“T18T21异常”列
data_q4['T18T21异常'] = np.where((data_q4['T18异常'] == 1) & (data_q4['T21异常'] == 1), 1, 0)

# 保存修改后的数据到新文件
data_q4.to_excel('女胎_标注后.xlsx', index=False)
data_q4

,序号,孕妇代码,年龄,身高,体重,末次月经,IVF妊娠,检测日期,检测抽血次数,检测孕周,...,生产次数,胎儿是否健康,怀孕时间,T13异常,T18异常,T21异常,二分类异常,T13T18异常,T13T21异常,T18T21异常
0,1,B001,32,162.0,82.0,2023-08-06 00:00:00,自然受孕,20231119,1,13w+5,...,0,是,96,0,0,0,0,0,0,0
1,2,B001,32,162.0,82.0,2023-08-06 00:00:00,自然受孕,20231208,2,17w+1,...,0,是,120,0,0,0,0,0,0,0
2,3,B001,32,162.0,85.0,2023-08-06 00:00:00,自然受孕,20240101,3,19w+6,...,0,是,139,0,0,0,0,0,0,0
3,4,B001,32,162.0,86.0,2023-08-06 00:00:00,自然受孕,20240123,4,23w,...,0,是,161,0,0,0,0,0,0,0
4,5,B002,33,165.0,96.0,2023-02-15 00:00:00,自然受孕,20230519,1,12w+5,...,0,是,89,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,601,B146,34,164.0,90.0,2022-12-31 00:00:00,自然受孕,20230615,4,23w+1,...,1,是,162,0,1,0,1,0,0,0
601,602,B147,32,166.0,86.0,2023-01-08 00:00:00,自然受孕,20230423,1,15w,...,1,是,105,0,0,0,0,0,0,0
602,603,B147,32,166.0,84.0,2023-01-08 00:00:00,自然受孕,20230513,2,15w+4,...,1,是,109,0,0,0,0,0,0,0
603,604,B147,32,166.0,86.0,2023-01-08 00:00:00,自然受孕,20230610,3,19w+4,...,1,是,137,0,0,0,0,0,0,0
